# Code Description:

In the following code, the closing price for each sector is represented by the average of the adjusted closing   prices of the constituent companies within that sector. Here's a breakdown of the steps involved in calculating this average closing price for each sector:

#### Data Collection:
- The code first fetches the list of S&P 500 companies and their respective sectors using the Wikipedia link provided.
- It then replaces specific tickers like 'BRK.B' with 'BRK-B' and 'BF.B' with 'BF-B' to ensure compatibility with Yahoo Finance.

#### Looping Through Sectors:
- The code then loops through each unique sector in the dataset.

#### Fetching Company Data:
- For each sector, it retrieves the adjusted closing prices of the constituent companies for the last 90 days from Yahoo Finance.

#### Calculating Average Closing Price:
It calculates the average closing price for each date by taking the mean of the adjusted closing prices for all companies within that sector on that date.

#### Data Formatting and Organization:
- The code organizes this data into a DataFrame with columns for 'Date', 'Sector', and 'Avg_Close' (average closing price).
- The DataFrame is pivoted to have sectors as columns and dates as rows.
- Finally, the DataFrame is stacked to have the sectors' average closing prices in a single column for each date.

#### Data Saving:
- The resulting DataFrame, representing the average closing prices for each sector, is saved to an Excel file.
- The calculated average closing prices for each sector represent an aggregate view of the performance of companies within that sector, giving an indication of the overall sector's stock price movement over the specified period (last 90 days in this case).

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sci_opt

from pprint import pprint
from sklearn.preprocessing import StandardScaler
# Set some display options for Pandas.
%config InlineBackend.figure_format ='retina'
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_columns', None)



In [ ]:
## Fetching S&P 500 Companies:
sp500_companies = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_companies['Symbol'] = sp500_companies['Symbol'].replace({'BRK.B': 'BRK-B', 'BF.B': 'BF-B'})

In [ ]:
sp500_companies

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [ ]:
# Dropping NaN values
sp500_companies.dropna(inplace=True)

In [ ]:
## Getting Unique Sectors and Dates:
unique_sectors = sp500_companies['GICS Sector'].unique()
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=90)).strftime('%Y-%m-%d')

In [ ]:
unique_sectors_df = pd.DataFrame(unique_sectors)
print(unique_sectors_df)

                         0
0              Industrials
1              Health Care
2   Information Technology
3         Consumer Staples
4                Utilities
5               Financials
6   Consumer Discretionary
7                Materials
8              Real Estate
9   Communication Services
10                  Energy


In [ ]:
number_of_sectors = len(unique_sectors)

In [ ]:
len(unique_sectors)

11

In [ ]:
## Fetching Historical Prices and Calculating Averages:
## The code fetches historical closing prices for each company in each sector and calculates the average closing price for the sector.
# Loop through each sector and download closing prices
all_avg_prices = []
all_dates = []
all_sectors = []
for sector in unique_sectors:
    companies = sp500_companies[sp500_companies['GICS Sector'] == sector]['Symbol'].tolist()
    prices = yf.download(companies, start=start_date, end=end_date)['Adj Close']
    avg_prices = prices.mean(axis=1)
    # Append the data
    all_avg_prices.extend(avg_prices.tolist())
    all_dates.extend([date.date() for date in prices.index])
    all_sectors.extend([sector] * len(avg_prices))

# Create a DataFrame
sp500_sectors_avg_prices = pd.DataFrame({
    'Date': all_dates,
    'Avg_Close': all_avg_prices,
    'Sector': all_sectors
})

#sp500_sectors_avg_prices = sp500_sectors_avg_prices.pivot(index='Date', columns='Sector', values='Avg_Close')
# Reverse the order of the DataFrame by dates
sp500_sectors_avg_prices = sp500_sectors_avg_prices[::-1]

print(sp500_sectors_avg_prices.head(100))

[*********************100%***********************]  77 of 77 completed
[*********************100%***********************]  64 of 64 completed
[*********************100%***********************]  64 of 64 completed
[*********************100%***********************]  38 of 38 completed
[*********************100%***********************]  30 of 30 completed
[*********************100%***********************]  72 of 72 completed
[*********************100%***********************]  53 of 53 completed
[*********************100%***********************]  29 of 29 completed
[*********************100%***********************]  31 of 31 completed
[*********************100%***********************]  22 of 22 completed
[*********************100%***********************]  23 of 23 completed
           Date   Avg_Close                  Sector
703  2023-11-10   85.216956                  Energy
702  2023-11-09   84.150001                  Energy
701  2023-11-08   84.473913                  Energy
700  2023-1

In [ ]:
sp500_sectors_avg_prices = sp500_sectors_avg_prices.pivot(
    index='Date',
    columns='Sector',
    values='Avg_Close'
)
print(sp500_sectors_avg_prices.head())

Sector      Communication Services  Consumer Discretionary  Consumer Staples     Energy  Financials  Health Care  Industrials  Information Technology   Materials  Real Estate  Utilities
Date                                                                                                                                                                                     
2023-08-14              108.676555              397.984067        108.685449  89.177779  145.606081   233.109744   188.342587              226.777679  129.620674   120.338698  65.667915
2023-08-15              107.854397              396.814123        107.475582  87.504414  143.117001   231.864662   186.132720              224.007215  127.666770   118.991642  64.635822
2023-08-16              106.748067              394.983365        107.157181  86.941313  143.137305   228.876733   185.061313              220.986992  126.822401   117.725655  64.954546
2023-08-17              105.246037              387.574859        105.

In [ ]:
sp500_sectors_avg_prices.dropna(inplace=True)

In [ ]:
print(sp500_sectors_avg_prices.isnull().sum())

Sector
Communication Services    0
Consumer Discretionary    0
Consumer Staples          0
Energy                    0
Financials                0
Health Care               0
Industrials               0
Information Technology    0
Materials                 0
Real Estate               0
Utilities                 0
dtype: int64


In [ ]:
sp500_sectors_avg_prices.to_excel('sp500_sectors_avg_prices.xlsx', index=True)

In [ ]:
# Calculate the Log of returns.
log_return = np.log(1 +sp500_sectors_avg_prices.pct_change().iloc[::-1])

# Drop rows with negative values
log_return = log_return[(log_return > 0).all(axis=1)]


# Generate Random Weights.
random_weights = np.array(np.random.random(number_of_sectors))

# Generate the Rebalance Weights, these should equal 1.
rebalance_weights = random_weights / np.sum(random_weights)


In [ ]:
print('Log Returns:')
print(log_return.head())

Log Returns:
Sector      Communication Services  Consumer Discretionary  Consumer Staples    Energy  Financials  Health Care  Industrials  Information Technology  Materials  Real Estate  Utilities
Date                                                                                                                                                                                   
2023-11-10                0.015518                0.019168          0.006965  0.012600    0.010749     0.009065     0.014786                0.027962   0.010486     0.010860   0.003447
2023-11-02                0.014616                0.021447          0.015604  0.028752    0.020801     0.019184     0.024089                0.022050   0.017851     0.029691   0.018812
2023-10-31                0.010750                0.004778          0.003464  0.006296    0.008070     0.009514     0.006770                0.013570   0.005497     0.018308   0.007343
2023-10-16                0.014209                0.011378         

In [ ]:
print((sp500_sectors_avg_prices == 0).sum())

Sector
Communication Services    0
Consumer Discretionary    0
Consumer Staples          0
Energy                    0
Financials                0
Health Care               0
Industrials               0
Information Technology    0
Materials                 0
Real Estate               0
Utilities                 0
dtype: int64


In [ ]:
log_return.to_excel('log_return.xlsx', index=True)

In [ ]:
# Calculate the percentage of negative log returns
percentage_negative_returns = (log_return < 0).mean().mean() * 100

print(f"Percentage of negative log returns: {percentage_negative_returns:.2f}%")

Percentage of negative log returns: 0.00%


In [ ]:

# Calculate the Expected Returns, annualize it by multiplying it by `252`.
risk_free_rate = .01
exp_ret = np.sum(((log_return.mean()-risk_free_rate) * rebalance_weights) * 252)

# Calculate the Expected Volatility, annualize it by multiplying it by `252`.
exp_vol = np.sqrt(
np.dot(
    rebalance_weights.T,
    np.dot(
        log_return.cov() * 252,
        rebalance_weights
    )
)
)

# Calculate the Sharpe Ratio.
sharpe_ratio = exp_ret / exp_vol

# Put the weights into a data frame to see them better.
weights_df = pd.DataFrame(data={
'random_weights': random_weights,
'rebalance_weights': rebalance_weights
})
print('')
print('='*80)
print('PORTFOLIO WEIGHTS:')
print('-'*80)
print(weights_df)
print('-'*80)

# Do the same with the other metrics.
metrics_df = pd.DataFrame(data={
    'Expected Portfolio Returns': exp_ret,
    'Expected Portfolio Volatility': exp_vol,
    'Portfolio Sharpe Ratio': sharpe_ratio
}, index=[0])

print('')
print('='*80)
print('PORTFOLIO METRICS:')
print('-'*80)
print(metrics_df)
print('-'*80)


PORTFOLIO WEIGHTS:
--------------------------------------------------------------------------------
    random_weights  rebalance_weights
0         0.969467           0.162267
1         0.727651           0.121792
2         0.391792           0.065577
3         0.359399           0.060155
4         0.615967           0.103099
5         0.983749           0.164657
6         0.060114           0.010062
7         0.350282           0.058629
8         0.697348           0.116720
9         0.689279           0.115370
10        0.129479           0.021672
--------------------------------------------------------------------------------

PORTFOLIO METRICS:
--------------------------------------------------------------------------------
   Expected Portfolio Returns  Expected Portfolio Volatility  Portfolio Sharpe Ratio
0                    0.303376                       0.078608                3.859359
--------------------------------------------------------------------------------


In [ ]:
# Initialize the components, to run a Monte Carlo Simulation.

# We will run 5000 iterations.
num_of_portfolios = 20000

# Prep an array to store the weights as they are generated, 5000 iterations for each of our 4 symbols.
all_weights = np.zeros((num_of_portfolios, number_of_sectors))

# Prep an array to store the returns as they are generated, 5000 possible return values.
ret_arr = np.zeros(num_of_portfolios)

# Prep an array to store the volatilities as they are generated, 5000 possible volatility values.
vol_arr = np.zeros(num_of_portfolios)

# Prep an array to store the sharpe ratios as they are generated, 5000 possible Sharpe Ratios.
sharpe_arr = np.zeros(num_of_portfolios)

# Start the simulations.
for ind in range(num_of_portfolios):

    # First, calculate the weights.
    weights = np.array(np.random.random(number_of_sectors))
    weights = weights / np.sum(weights)

    # Add the weights, to the `weights_arrays`.
    all_weights[ind, :] = weights

    # Calculate the expected log returns, and add them to the `returns_array`.
    ret_arr[ind] = np.sum(((log_return.mean()-risk_free_rate) * weights) * 252)

    # Calculate the volatility, and add them to the `volatility_array`.
    vol_arr[ind] = np.sqrt(
        np.dot(weights.T, np.dot(log_return.cov() * 252, weights)))


    # Calculate the Sharpe Ratio and Add it to the `sharpe_ratio_array`.
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

# Let's create our "Master Data Frame", with the weights, the returns, the volatility, and the Sharpe Ratio
simulations_data = [ret_arr, vol_arr, sharpe_arr, all_weights]

# Create a DataFrame from it, then Transpose it so it looks like our original one.
simulations_df = pd.DataFrame(data=simulations_data).T

# Give the columns the Proper Names.
simulations_df.columns = [
    'Returns',
    'Volatility',
    'Sharpe Ratio',
    'Portfolio Weights'
]

# Make sure the data types are correct, we don't want our floats to be strings.
simulations_df = simulations_df.infer_objects()

# Print out the results.
print('')
print('='*80)
print('SIMULATIONS RESULT:')
print('-'*80)

# Print PORTFOLIO WEIGHTS
print('='*80)
print('PORTFOLIO WEIGHTS:')
print('-'*80)
print(simulations_df['Portfolio Weights'].head())
print('-'*80)

# Print PORTFOLIO METRICS
print('='*80)
print('PORTFOLIO METRICS:')
print('-'*80)
print(simulations_df[['Returns', 'Volatility', 'Sharpe Ratio']].head())
print('-'*80)


SIMULATIONS RESULT:
--------------------------------------------------------------------------------
PORTFOLIO WEIGHTS:
--------------------------------------------------------------------------------
0      [0.0918919289282305, 0.0805512602001684, 0.06796558842335781, 0.14813355100074269, 0.09183784811855765, 0.07064585627303377, 0.003819480025541663, 0.13017929275117268, 0.16581460724482935, 0.07437150207875824, 0.07478908495560714]
1    [0.029018849256720287, 0.05865935398760163, 0.041007339339173186, 0.10271748271488208, 0.06207686210870877, 0.0927944808505549, 0.17336563482730946, 0.20739360760718917, 0.07733571318541398, 0.11782164271837482, 0.03780903340407175]
2     [0.10587252256543288, 0.09201555577209798, 0.08853833955704861, 0.05760756520127283, 0.021094809610149277, 0.10429100055164979, 0.09896980558936067, 0.13984522542720093, 0.09544997558484104, 0.1626136042241218, 0.03370159591682422]
3     [0.08344720403608406, 0.004328535619110992, 0.05681531421692042, 0.03207574367

In [ ]:
# Return the Max Sharpe Ratio from the run.
max_sharpe_ratio = simulations_df.loc[simulations_df['Sharpe Ratio'].idxmax()]

# Return the Min Volatility from the run.
min_volatility = simulations_df.loc[simulations_df['Volatility'].idxmin()]

print('')
print('='*80)
print('MAX SHARPE RATIO:')
print('-'*80)
print(max_sharpe_ratio)
print('-'*80)

print('')
print('='*80)
print('MIN VOLATILITY:')
print('-'*80)
print(min_volatility)
print('-'*80)



MAX SHARPE RATIO:
--------------------------------------------------------------------------------
Returns                                                                                                                                                                                                                                            0.619264
Volatility                                                                                                                                                                                                                                         0.083392
Sharpe Ratio                                                                                                                                                                                                                                       7.425931
Portfolio Weights    [0.18101971080851087, 0.04580935876423305, 0.0104916310683585, 0.04366286931897253, 0.004320781789587477, 0.027819328912733

# MAX SHARPE RATIO Portfolio:
- Max Sharpe Ratio Portfolio:
- Returns: The expected return of the portfolio is 61.93%.
- Volatility: The volatility (risk) of the portfolio is 8.34%.

### Sharpe Ratio: A Sharpe Ratio of 7.43 is quite high, indicating a good balance between return and risk.

### Portfolio Weights:

The portfolio is distributed across different assets, with the highest weight (22.73%) on the 8th asset.
The weights suggest a diversified allocation, and the optimization process has determined the allocation that maximizes the Sharpe Ratio for the given assets.

# Min Volatility Portfolio:
- Returns: The expected return of the portfolio is 11.62%.
- Volatility: The volatility (risk) of the portfolio is 6.10%.
- Sharpe Ratio: The Sharpe Ratio is 1.90, which is lower than the max Sharpe Ratio portfolio, but still provides a measure of risk-adjusted return.

### Portfolio Weights:

The portfolio is again diversified, with the highest weight (26.83%) on the 1st asset and significant weights on the 3rd and 11th assets.
The weights are determined to minimize the volatility of the portfolio, resulting in a more conservative allocation.

In summary, the first portfolio aims to maximize the risk-adjusted return (Sharpe Ratio) and has a higher expected return but also higher volatility. The second portfolio aims to minimize volatility with a more conservative allocation but with a lower expected return. The choice between the two depends on the investor's risk tolerance and investment objectives.

### Portfolio Weights:

#### Max Sharpe Ratio Portfolio:

#### Sector Weights:
- Sector 1: 18.10%
- Sector 2: 4.58%
- Sector 3: 1.05%
- Sector 4: 4.37%
- Sector 5: 0.43%
- Sector 6: 2.78%
- Sector 7: 6.19%
- Sector 8: 22.73%
- Sector 9: 14.14%
- Sector 10: 20.24%
- Sector 11: 5.39%

#### Min Volatility Portfolio:

#### Sector Weights:
- Sector 1: 26.83%
- Sector 2: 2.88%
- Sector 3: 23.50%
- Sector 4: 5.97%
- Sector 5: 0.70%
- Sector 6: 7.63%
- Sector 7: 4.40%
- Sector 8: 3.56%
- Sector 9: 1.41%
- Sector 10: 3.40%
- Sector 11: 19.73%

These percentages represent the allocation of each portfolio to different sectors. It's a useful breakdown to understand the diversification within each portfolio and can be valuable for investors who want to ensure a balanced exposure to different sectors or industries.

In [ ]:
# DICTIONARIES WITH WEIGHTS

maxsharpedict = { "Communication Services": 0.181, "Consumer Discretionary": 0.0458,
                  "Consumer Staples": 0.0105, "Energy": 0.0437, "Financials": 0.0043,
                  "Health Care": 0.0278, "Industrials": 0.0619, "Information Technology": 0.2273,
                  "Materials": 0.1414, "Real Estate": 0.2024,"Utilites": 0.0539 }

minvoldict = { "Communication Services": 0.2683, "Consumer Discretionary": 0.0288,
               "Consumer Staples": 0.235, "Energy": 0.0597, "Financials": 0.007,
               "Health Care": 0.0763, "Industrials": 0.044, "Information Technology": 0.0356,
               "Materials": 0.0141, "Real Estate": 0.034, "Utilites": 0.1973 }

In [ ]:
# MACHINE LEARNING COMPONENT

'''
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

print("Unique values in 'Portfolio Weights' column:")
print(simulations_df["Portfolio Weights"].unique())

simulations_df["Portfolio Weights"] = simulations_df["Portfolio Weights"].apply(lambda x: np.concatenate(x) if isinstance(x, list) else x)
X = simulations_df[["Volatility", "Sharpe Ratio"]]
y = np.concatenate(simulations_df["Portfolio Weights"].to_numpy())

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
'''